In [1]:
from pymongo import MongoClient
import pandas as pd
import folium
import branca
from bson import ObjectId

In [2]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [3]:
db, coll = connectCollection('companies','design_new')

In [4]:
#funcion que nos dice lo que hay cerca de companies
def getNearby(lat,lng,comptype,radius=300):
    db, coll = connectCollection('companies',comptype)
    results = list(coll.find(
        {"coords":
         {"$near":
          {"$geometry":
           {
               "type":"Point",
               "coordinates":[lng,lat]
           },
           "$maxDistance":radius
          }
         }
        }
    ))
    total = len(results)
    return results if total > 0 else None

getNearby(37.4374154,-122.1722525,"party")

In [5]:
design_comp = list(coll.find({}))
venues=['party','schools','starbucks','vegan']
companies_nearby = []
for company in design_comp:
    long,lat = company['coords']['coordinates']
    companies_nearby.append({company["_id"]:{poi:getNearby(lat,long,poi) for poi in venues}})

In [6]:
len(companies_nearby)

37

In [8]:
#buscamos las oficinas que tengan los cuatro requerimientos 
final_companies=[]
for company in companies_nearby:
    for c in company.values():
        if c['party'] is not None and c['schools'] is not None and c['vegan'] is not None and c['starbucks'] is not None:
            print('ok')
            final_companies.append(company)

ok


In [9]:
selected=final_companies[0]

In [22]:
map_city=folium.Map([-122.1615958214, 37.4438754674][::-1], zoom_start=16)
folium.Marker(final_comp[0]['coords']['coordinates'][::-1],
                    radius=1,
                    icon=folium.Icon(icon='briefcase',color='orange'),
                    tooltip ='Our Company'
                   ).add_to(map_city)
for star in selected[ObjectId('5dd6b0add27a078c8c61cb69')]['starbucks']:
    folium.Marker(star['coords']['coordinates'][::-1],
                    radius=1,
                    icon=folium.Icon(icon='star',color='red'), 
                    tooltip ='Coffee: '+star['name']
                   ).add_to(map_city)

for party in selected[ObjectId('5dd6b0add27a078c8c61cb69')]['party']:
    folium.Marker(party['coords']['coordinates'][::-1],
                    radius=1,
                    icon=folium.Icon(icon='glass',color='blue'), 
                    tooltip ='Nightclub: '+party['name']
                   ).add_to(map_city)

for school in selected[ObjectId('5dd6b0add27a078c8c61cb69')]['schools']:
    folium.Marker(school['coords']['coordinates'][::-1],
                    radius=1,
                    icon=folium.Icon(icon='education',color='darkpurple'), 
                    tooltip="School: "+school['name']
                   ).add_to(map_city)

for vegan in selected[ObjectId('5dd6b0add27a078c8c61cb69')]['vegan']:
    folium.Marker(vegan['coords']['coordinates'][::-1],
                    radius=1,
                    icon=folium.Icon(icon='cutlery',color='green'), 
                    tooltip="Vegan food: "+vegan['name']
                   ).add_to(map_city)

In [23]:
map_city